In [1]:
from functions import *
from models import *
from datasets import *

%load_ext autoreload
%autoreload 2

c:\Users\user\AppData\Local\Programs\Orange\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Считаем данные:

In [2]:
data = pd.read_csv('data_lagged_features_version2.csv')

In [3]:
data

,Unnamed: 0,period_dt,NUM_CONSULTANT,PRICE_AFTER_DISC,PRICE_REGULAR,PRODUCT_LVL_RK6,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK2,STORE_LOCATION_LVL_RK3,...,demand_lag71d_alpha14_keyPRODUCT_LVL_RK6_STORE_LOCATION_LVL_RK4_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag71d_alpha42_keyPRODUCT_LVL_RK6_STORE_LOCATION_LVL_RK4_preagsum_PROMO1_FLAG_dynamic_ewm,demand_lag71d_w14D_keyPRODUCT_LVL_RK6_STORE_LOCATION_LVL_RK4_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag71d_w28D_keyPRODUCT_LVL_RK6_STORE_LOCATION_LVL_RK4_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag71d_w35D_keyPRODUCT_LVL_RK6_STORE_LOCATION_LVL_RK4_preagsum_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag71d_alpha14_keyPRODUCT_LVL_RK6_STORE_LOCATION_LVL_RK4_preagmean_PROMO1_FLAG_dynamic_ewm,demand_lag71d_alpha42_keyPRODUCT_LVL_RK6_STORE_LOCATION_LVL_RK4_preagmean_PROMO1_FLAG_dynamic_ewm,demand_lag71d_w14D_keyPRODUCT_LVL_RK6_STORE_LOCATION_LVL_RK4_preagmean_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag71d_w28D_keyPRODUCT_LVL_RK6_STORE_LOCATION_LVL_RK4_preagmean_agmean_PROMO1_FLAG_dynamic_rolling,demand_lag71d_w35D_keyPRODUCT_LVL_RK6_STORE_LOCATION_LVL_RK4_preagmean_agmean_PROMO1_FLAG_dynamic_rolling
0,0,2016-12-19,0.0,1899.000000,1899.0,6198,1.0,0.0,10,203,...,1.934093e-18,0.000002,0.0,0.0,0.0,0.397846,0.265790,NaN,NaN,NaN
1,1,2016-12-19,0.0,1374.500000,2749.0,113176,1.0,0.0,10,203,...,9.958647e-18,0.000014,0.0,0.0,0.0,0.120130,0.114264,NaN,NaN,NaN
2,2,2016-12-19,0.0,2749.000000,2749.0,113176,1.0,0.0,10,203,...,9.958647e-18,0.000014,0.0,0.0,0.0,0.120130,0.114264,NaN,NaN,NaN
3,3,2016-12-19,0.0,824.500000,1649.0,113176,1.0,0.0,10,203,...,9.958647e-18,0.000014,0.0,0.0,0.0,0.120130,0.114264,NaN,NaN,NaN
4,4,2016-12-19,0.0,708.500000,1417.0,113176,1.0,0.0,10,203,...,9.958647e-18,0.000014,0.0,0.0,0.0,0.120130,0.114264,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550139,550139,2019-12-30,0.0,584.700000,1949.0,113176,1.0,0.0,10,189,...,2.711663e-17,0.000020,0.0,0.0,0.0,0.120130,0.114264,NaN,NaN,NaN
550140,550140,2019-12-30,0.0,1272.385714,2699.0,113176,1.0,0.0,10,189,...,2.711663e-17,0.000020,0.0,0.0,0.0,0.120130,0.114264,NaN,NaN,NaN
550141,550141,2019-12-30,0.0,796.700000,1799.0,113176,1.0,0.0,10,189,...,2.711663e-17,0.000020,0.0,0.0,0.0,0.120130,0.114264,NaN,NaN,NaN
550142,550142,2019-12-30,5.0,745.200000,1449.0,9984,1.0,0.0,10,203,...,2.415581e-17,0.000016,0.0,0.0,0.0,0.330494,0.304801,NaN,NaN,NaN


Предобработка:

In [4]:
data_lagged_features = data.copy()

In [5]:
test_real = pd.read_csv("lt_test.csv", sep=',')

In [6]:
df_train, df_test_m, target_mean, target_stdev, features = preprocess(data_lagged_features, test_real, target='demand', id_cols=['product_id', 'location_id'], date_col='period_dt', scl=0)

In [7]:
df_train.drop(columns=['Unnamed: 0'], inplace=True)
df_test_m.drop(columns=['Unnamed: 0'], inplace=True)

In [142]:
X_train = df_train.copy()
df_test = df_test_m.copy()

Создадим новые признаки и обрежем данные:

In [143]:
X_train['period_dt'] = pd.to_datetime(X_train['period_dt'])
first_arrival = X_train.groupby(['product_id', 'location_id'])['period_dt'].min().reset_index()
first_arrival.rename(columns={'period_dt': 'arrival'}, inplace=True)
X_train = X_train.merge(first_arrival, on=['product_id', 'location_id'])

In [144]:
X_train['period_dt'] = pd.to_datetime(X_train['period_dt'])
X_train['arrival'] = pd.to_datetime(X_train['arrival'])

X_train['days_diff'] = (X_train['period_dt'] - X_train['arrival']).dt.days
X_train['is2w'] = X_train['days_diff'].apply(lambda x: 1 if x <= 110 else 0)
X_train.drop(columns=['days_diff', 'arrival'], inplace=True)

In [145]:
X_train = X_train[X_train['is2w'] != 0]

In [146]:
X_train["holiday_flag"] = ((X_train['ind_of_month'] == 3) & (X_train['ind_of_day'] >= 1)) | ((X_train['ind_of_month'] == 3) & (X_train['ind_of_day'] <= 8))
X_train["holiday_flag2"] = ((X_train['ind_of_month'] == 12) & (X_train['ind_of_day'] >= 20)) | ((X_train['ind_of_month'] == 12) & (X_train['ind_of_day'] <= 31))
X_train["holiday_flag"] = X_train["holiday_flag"].astype(int)
X_train["holiday_flag2"] = X_train["holiday_flag2"].astype(int)

In [147]:
df_test["holiday_flag"] = ((df_test['ind_of_month'] == 3) & (df_test['ind_of_day'] >= 1)) | ((df_test['ind_of_month'] == 3) & (df_test['ind_of_day'] <= 8))
df_test["holiday_flag2"] = ((df_test['ind_of_month'] == 12) & (df_test['ind_of_day'] >= 20)) | ((df_test['ind_of_month'] == 12) & (df_test['ind_of_day'] <= 31))
df_test["holiday_flag"] = df_test["holiday_flag"].astype(int)
df_test["holiday_flag2"] = df_test["holiday_flag2"].astype(int)

In [148]:
X_train["discount"] = X_train["PRICE_REGULAR"] / X_train["PRICE_AFTER_DISC"]

In [149]:
df_test["discount"] = df_test["PRICE_REGULAR"] / df_test["PRICE_AFTER_DISC"]

In [150]:
unique_counts = X_train.nunique()
zero_counts = (X_train == 0).sum()
nan_counts = X_train.isnull().sum()

total_rows = len(X_train)
zero_percentage = zero_counts / total_rows * 100
nan_percentage = nan_counts / total_rows * 100

In [151]:
pd.options.display.max_rows = 110

summary_stats = pd.DataFrame({
    'Уникальные значения': unique_counts,
    'Нулевые значения': zero_counts,
    'Доля нулевых значений %': zero_percentage,
    'Пустые значения': nan_counts,
    'Доля пустых значений %': nan_percentage,
    'Тип данных': X_train.dtypes
})

summary_stats

,Уникальные значения,Нулевые значения,Доля нулевых значений %,Пустые значения,Доля пустых значений %,Тип данных
period_dt,122,0,0.000000,0,0.000000,datetime64[ns]
NUM_CONSULTANT,8,86526,93.339806,0,0.000000,float64
PRICE_AFTER_DISC,2374,0,0.000000,0,0.000000,float64
PRICE_REGULAR,652,0,0.000000,0,0.000000,float64
PRODUCT_LVL_RK6,17,0,0.000000,0,0.000000,int64
PROMO1_FLAG,3,35456,38.248112,0,0.000000,float64
PROMO2_FLAG,2,90912,98.071197,0,0.000000,float64
STORE_LOCATION_LVL_RK2,1,0,0.000000,0,0.000000,int64
STORE_LOCATION_LVL_RK3,13,0,0.000000,0,0.000000,int64
STORE_LOCATION_LVL_RK4,13,0,0.000000,0,0.000000,int64


Выберем наши признаки:

In [152]:
columns_to_exclude = ['is2w', 'discount', 'ind_of_year', 'ind_of_day', 'PRICE_AFTER_DISC', 
                      'weekday', 'NUM_CONSULTANT', 'is_weekend', 'NoFilter', 'period_dt']

features_to_learn_df = X_train.drop(columns=columns_to_exclude, axis=1)

threshold_nan = 30
features_to_learn_df = features_to_learn_df.loc[:, features_to_learn_df.isnull().sum() <= threshold_nan]

features_to_learn = features_to_learn_df.columns

features_to_learn

Index(['PRICE_REGULAR', 'PRODUCT_LVL_RK6', 'PROMO1_FLAG', 'PROMO2_FLAG',
       'STORE_LOCATION_LVL_RK2', 'STORE_LOCATION_LVL_RK3',
       'STORE_LOCATION_LVL_RK4', 'demand', 'location_id', 'product_id', 'id',
       'monthday', 'ind_of_month', 'holiday_flag', 'holiday_flag2',
       'demand_lag53d_alpha14_keyPRODUCT_LVL_RK6_STORE_LOCATION_LVL_RK4_preagsum_PROMO1_FLAG_dynamic_ewm',
       'demand_lag53d_alpha42_keyPRODUCT_LVL_RK6_STORE_LOCATION_LVL_RK4_preagsum_PROMO1_FLAG_dynamic_ewm',
       'demand_lag61d_alpha14_keyPRODUCT_LVL_RK6_STORE_LOCATION_LVL_RK4_preagsum_PROMO1_FLAG_dynamic_ewm',
       'demand_lag61d_alpha42_keyPRODUCT_LVL_RK6_STORE_LOCATION_LVL_RK4_preagsum_PROMO1_FLAG_dynamic_ewm',
       'demand_lag71d_alpha14_keyPRODUCT_LVL_RK6_STORE_LOCATION_LVL_RK4_preagsum_PROMO1_FLAG_dynamic_ewm',
       'demand_lag71d_alpha42_keyPRODUCT_LVL_RK6_STORE_LOCATION_LVL_RK4_preagsum_PROMO1_FLAG_dynamic_ewm'],
      dtype='object')

In [153]:
X_train = X_train[features_to_learn]

In [154]:
mean_values = X_train.drop(columns=['demand']).mean()
X_train = X_train.fillna(mean_values)

In [155]:
y_train = X_train['demand']
X_train = X_train.drop(['id', 'demand'], axis=1)

Обучаем модель:

In [156]:
from sklearn.metrics import mean_squared_error
from sklego.meta import ZeroInflatedRegressor
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder

In [157]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.01, random_state=1)

In [158]:
ohe = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
categorical_features = ['location_id', 'STORE_LOCATION_LVL_RK2', 'STORE_LOCATION_LVL_RK3', 'STORE_LOCATION_LVL_RK4', 'PRODUCT_LVL_RK6']

X_train_encoded = ohe.fit_transform(X_train[categorical_features])
X_test_encoded = ohe.transform(X_test[categorical_features])

X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=ohe.get_feature_names_out(categorical_features))
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=ohe.get_feature_names_out(categorical_features))

X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

X_train = pd.concat([X_train.drop(categorical_features, axis=1), X_train_encoded_df], axis=1)
X_test = pd.concat([X_test.drop(categorical_features, axis=1), X_test_encoded_df], axis=1)

classifier = XGBClassifier(random_state=1, max_depth=4, learning_rate=0.05, reg_alpha=700, reg_lambda=700)

xgb_regressor = xgb.XGBRegressor(random_state=1, max_depth=4, learning_rate=0.05, reg_alpha=700, reg_lambda=700)

zir = ZeroInflatedRegressor(classifier=classifier, regressor=xgb_regressor)

zir.fit(X_train, y_train)

y_pred = zir.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 0.5887212548490899


In [170]:
demand = y_train

bins = [-1, 0.5, 1, 2, 3, 5, 100]
labels = ['0-0.5', '0.5-1', '1-2', '2-3', '3-5', '5-100']

demand_categories = pd.cut(demand, bins=bins, labels=labels, include_lowest=True)

category_counts = demand_categories.value_counts().sort_index()

for category, count in category_counts.items():
    print(f"Категория: {category}, Количество: {count}")

Категория: 0-0.5, Количество: 77284
Категория: 0.5-1, Количество: 9474
Категория: 1-2, Количество: 3280
Категория: 2-3, Количество: 753
Категория: 3-5, Количество: 553
Категория: 5-100, Количество: 429


Сделаем subbmit:

In [160]:
features_to_learn = X_train.columns

In [161]:
mean_values = df_test.drop(columns=['demand']).mean()
df_test = df_test.fillna(mean_values)
y_results = df_test[['id', 'demand']]

In [162]:
ohe = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
df_test_encoded = ohe.fit_transform(df_test[categorical_features])

df_test_encoded_df = pd.DataFrame(df_test_encoded, columns=ohe.get_feature_names_out(categorical_features))

df_test.reset_index(drop=True, inplace=True)

df_test = pd.concat([df_test.drop(categorical_features, axis=1), df_test_encoded_df], axis=1)

In [163]:
df_test = df_test[features_to_learn.intersection(df_test.columns)]
X_test = df_test.copy()
missing_features = set(features_to_learn) - set(df_test.columns)
for feature in missing_features:
    X_test[feature] = 0
y_pred_res = zir.predict(X_test)
y_results['demand'] = y_pred_res
y_results.loc[y_results['demand'] < 0, ['demand']] = 0.0
y_results.loc[y_results['demand'] < 0, ['demand']]

,demand


In [164]:
y_results = y_results.sort_values(by='id')

In [165]:
y_results

,id,demand
382946,0,0.000000
384100,1772,0.000000
389921,1773,0.000000
395635,1774,0.000000
400615,1775,0.000000
...,...,...
544630,550139,1.261072
549930,550140,0.749981
545331,550141,1.261072
547380,550142,0.749981


In [168]:
demand = y_results.demand

bins = [-1, 0.5, 1, 2, 3, 5]
labels = ['0-0.5', '0.5-1', '1-2', '2-3', '3-5']

demand_categories = pd.cut(demand, bins=bins, labels=labels, include_lowest=True)

category_counts = demand_categories.value_counts().sort_index()

for category, count in category_counts.items():
    print(f"Категория: {category}, Количество: {count}")

Категория: 0-0.5, Количество: 32824
Категория: 0.5-1, Количество: 776
Категория: 1-2, Количество: 501
Категория: 2-3, Количество: 6
Категория: 3-5, Количество: 0


In [167]:
y_results[['id', 'demand']].to_csv('final_subbmit_delta.csv', sep=',', encoding='utf-8', index=False)